In [39]:
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from pandas_datareader import data as pdr
import yfinance as yf

In [4]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

In [43]:
%run utils.ipynb

In [6]:
new_articles = pd.read_parquet('/Users/karan_mudaliar/Desktop/CS 7290/Project/Data/financial-news-data/financial_data.parquet.gzip')
new_articles.head()

,Headline,Journalists,Date,Link,Article
0,"Hitachi, GE boost alliance in nuclear power bu...",[],"Mon Nov 13, 2006 3:16am EST",http://www.reuters.com/article/2006/11/13/us-e...,TOKYO (Reuters) - Hitachi Ltd. ( 6501.T ) sai...
1,"Volvo to cut 1,000 staff at Virginia plant",[],"Mon Nov 13, 2006 8:45am EST",http://www.reuters.com/article/2006/11/13/us-a...,STOCKHOLM (Reuters) - Truck maker Volvo said ...
2,European banks hiding full pension obligations,[Andrew Hurst],"Mon Nov 13, 2006 3:15am EST",http://www.reuters.com/article/2006/11/13/us-f...,"ZURICH, Nov 13 (Reuter) - West European banks ..."
3,"Hitachi, GE to form joint nuclear power ventures",[Mayumi Negishi],"Mon Nov 13, 2006 7:13am EST",http://www.reuters.com/article/2006/11/13/us-e...,TOKYO (Reuters) - Japan's Hitachi Ltd. and U....
4,Eddie Bauer agrees to be bought for $286 million,[],"Mon Nov 13, 2006 7:29am EST",http://www.reuters.com/article/2006/11/13/us-r...,(Reuters) - Eddie Bauer Holdings Inc. EBHI.O s...


In [7]:
stock_info_df = pd.read_csv('/Users/karan_mudaliar/Desktop/CS 7290/Project/Data/stock_info.csv')
stock_info_df.info

<bound method DataFrame.info of      Ticker                     Name Exchange
0         A     Agilent Technologies     NYSE
1        AA               Alcoa Inc.     NYSE
2       AAN              Aaron's Inc     NYSE
3       AAT    American Assets Trust     NYSE
4       AAV  Advantage Oil & Gas Ltd     NYSE
...     ...                      ...      ...
5777     ZN       Zion Oil & Gas Inc   NASDAQ
5778   ZNGA               Zynga Inc.   NASDAQ
5779   ZOLT         Zoltek Companies   NASDAQ
5780   ZOOM        Zoom Technologies   NASDAQ
5781   ZUMZ              Zumiez Inc.   NASDAQ

[5782 rows x 3 columns]>

In [8]:
import pandas as pd
import dask.dataframe as dd
from dask.multiprocessing import get

In [16]:
ddf = dd.from_pandas(stock_info_df, npartitions=30)

In [17]:
ddf['About'] = ddf.map_partitions(lambda df: df['Name'].apply(fetch_description), meta=('Name', 'object'))

# Compute the result to get back a pandas DataFrame
result_df = ddf.compute()

# Display the updated DataFrame
print(result_df)

     Ticker                     Name Exchange  \
0         A     Agilent Technologies     NYSE   
1        AA               Alcoa Inc.     NYSE   
2       AAN              Aaron's Inc     NYSE   
3       AAT    American Assets Trust     NYSE   
4       AAV  Advantage Oil & Gas Ltd     NYSE   
...     ...                      ...      ...   
5777     ZN       Zion Oil & Gas Inc   NASDAQ   
5778   ZNGA               Zynga Inc.   NASDAQ   
5779   ZOLT         Zoltek Companies   NASDAQ   
5780   ZOOM        Zoom Technologies   NASDAQ   
5781   ZUMZ              Zumiez Inc.   NASDAQ   

                                                  About  
0     Agilent Technologies is a global leader in lif...  
1     Alcoa Inc. is a global leader in lightweight m...  
2     Aaron's Inc. is a leading omnichannel provider...  
3     American Assets Trust is a real estate investm...  
4     Advantage Oil & Gas Ltd. is a Canadian company...  
...                                                 ...  
5777 

In [20]:
result_df.to_csv ('/Users/karan_mudaliar/Desktop/CS 7290/Project/Data/stock_info_description.csv', index = None, header=True) 

In [14]:
result_df = pd.read_csv('/Users/karan_mudaliar/Desktop/CS 7290/Project/Data/stock_info_description.csv')
result_df.head()

,Ticker,Name,Exchange,About
0,A,Agilent Technologies,NYSE,Agilent Technologies is a global leader in lif...
1,AA,Alcoa Inc.,NYSE,Alcoa Inc. is a global leader in lightweight m...
2,AAN,Aaron's Inc,NYSE,Aaron's Inc. is a leading omnichannel provider...
3,AAT,American Assets Trust,NYSE,American Assets Trust is a real estate investm...
4,AAV,Advantage Oil & Gas Ltd,NYSE,Advantage Oil & Gas Ltd. is a Canadian company...


In [ ]:
ddf = dd.from_pandas(result_df, npartitions=100)  # Adjust npartitions based on your system and dataset size

meta = pd.DataFrame({
    'Ticker': pd.Series([], dtype='object'),
    'Name': pd.Series([], dtype='object'),
    'Exchange': pd.Series([], dtype='object'),
    'About': pd.Series([], dtype='object'),
    'Encoded_About': pd.Series([], dtype='object')  # Adjust dtype if your embeddings are not stored as objects
})

# Use Dask's map_partitions to apply the function to each partition in parallel
encoded_ddf = ddf.map_partitions(generate_embeddings, meta=meta)

# Compute the result to get back a pandas DataFrame
encoded_df = encoded_ddf.compute()

In [19]:
encoded_df.to_csv ('/Users/karan_mudaliar/Desktop/CS 7290/Project/Data/stock_info_description_embedded.csv', index = None, header=True) 

In [31]:
encoded_df = pd.read_csv('/Users/karan_mudaliar/Desktop/CS 7290/Project/Data/stock_info_description_embedded.csv')
encoded_df

,Ticker,Name,Exchange,About,Encoded_About
0,A,Agilent Technologies,NYSE,Agilent Technologies is a global leader in lif...,[-0.3952235 -0.09160021 -0.18406898 ... 0.26...
1,AA,Alcoa Inc.,NYSE,Alcoa Inc. is a global leader in lightweight m...,[-0.07310738 0.5146998 -0.09491941 ... -0.48...
2,AAN,Aaron's Inc,NYSE,Aaron's Inc. is a leading omnichannel provider...,[-0.45554543 -0.06957607 -0.8179232 ... -0.66...
3,AAT,American Assets Trust,NYSE,American Assets Trust is a real estate investm...,[-0.39959103 1.6209936 0.02285 ... -0.60...
4,AAV,Advantage Oil & Gas Ltd,NYSE,Advantage Oil & Gas Ltd. is a Canadian company...,[-0.26057047 0.97616833 -0.7055656 ... -0.94...
...,...,...,...,...,...
5777,ZN,Zion Oil & Gas Inc,NASDAQ,Zion Oil & Gas Inc. is a publicly traded oil a...,[-0.272786 0.29430646 0.04780815 ... -0.66...
5778,ZNGA,Zynga Inc.,NASDAQ,Zynga Inc. is a leading developer of social ga...,[ 0.30343133 0.17543581 0.20114104 ... 0.08...
5779,ZOLT,Zoltek Companies,NASDAQ,"Zoltek Companies, Inc. is a leading producer o...",[-0.01144433 0.24722561 -0.2640686 ... -0.02...
5780,ZOOM,Zoom Technologies,NASDAQ,Zoom Technologies is a leading provider of vid...,[ 0.05813441 -0.2777733 -0.35044852 ... -0.18...


In [63]:
encoded_df[:10]['Encoded_About']

0    [-0.3952235  -0.09160021 -0.18406898 ...  0.26...
1    [-0.07310738  0.5146998  -0.09491941 ... -0.48...
2    [-0.45554543 -0.06957607 -0.8179232  ... -0.66...
3    [-0.39959103  1.6209936   0.02285    ... -0.60...
4    [-0.26057047  0.97616833 -0.7055656  ... -0.94...
5    [-0.22920597  1.0093793  -0.32742697 ...  0.10...
6    [-0.41854858  0.29937333 -0.2567882  ... -0.60...
7    [ 0.06114767 -0.02294435  0.2302519  ...  0.00...
8    [-0.29704496  0.27340218 -0.46518877 ... -1.07...
9    [-5.6852028e-04  7.3149818e-01  2.3919366e-01 ...
Name: Encoded_About, dtype: object

In [66]:
import pandas as pd

# Assuming 'encoded_df' is your DataFrame and 'Encoded_About' contains the vectors as strings

# Function to convert space-separated numbers in a string to comma-separated
def convert_to_comma_separated(vector_str):
    # Trim the brackets if present
    vector_str = vector_str.strip('[]')
    # Replace sequences of spaces with a single comma
    comma_separated_str = ','.join(vector_str.split())
    # Add brackets back if needed
    return f'[{comma_separated_str}]'

# Apply the function to the 'Encoded_About' column
encoded_df['Encoded_About'] = encoded_df['Encoded_About'].apply(convert_to_comma_separated)

encoded_df.to_csv ('/Users/karan_mudaliar/Desktop/CS 7290/Project/Data/stock_info_description_embedded.csv', index = None, header=True) 
